Import Libs

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# own packages
from preprocess_data import clean_text
from preprocess_data import lemmatize_text
from preprocess_data import tfidf_vec_transform, tfidf_vec_fit_transform
from preprocess_data import count_vectorizer
from models import model_sfnn_train
from models import model_rf_train, model_naives_bayes, model_logistic_regression, model_multinominalNB_train, predict_values



Function for reading the data

In [2]:
# get csv data
def read_Data(path):

    # Define column names
    column_names = ['label', 'text']

    df = pd.read_csv(path, delimiter='\t', encoding="utf-8-sig", header=None, names=column_names)
    df.columns = df.columns.str.replace('\ufeff', '')
    df['label'] = df['label'].astype(str).str.replace('\ufeff', '')
    df['label'] = df['label'].astype(int)
    
    return df



In [3]:
## read data windows
data_train_val = read_Data(r".\data\training_data_lowercase.csv")           # used for train and val data
data_test = read_Data(r".\data\testing_data_lowercase_nolabels.csv")        # data for the predictions

## read data Mac
#data_train_val = read_Data("data/training_data_lowercase.csv")           # used for train and val data
#data_test = read_Data("data/testing_data_lowercase_nolabels.csv")        # data for the predictions


## split data_train into train and val data
data_train, data_val = train_test_split(data_train_val, test_size=0.2, random_state=42, stratify=data_train_val['label'])


In [4]:
## preview data
#print(data_train.head, "\n")
#print(data_test.head, "\n")
print("Shape train data:\n", data_train_val.shape)
print("Shape test data:\n", data_test.shape)

# clean data
data_train['cleaned_text'] = data_train['text'].apply(clean_text)
data_val['cleaned_text'] = data_val['text'].apply(clean_text)
data_test['cleaned_text'] = data_test['text'].apply(clean_text)

print("Cleand text train: \n", data_train["cleaned_text"].head, "\n")
print("Cleand text test: \n", data_test["cleaned_text"].head, "\n")


## lemmatize data
data_train['lemmatized_text'] = data_train['cleaned_text'].apply(lemmatize_text)
data_val['lemmatized_text'] = data_val['cleaned_text'].apply(lemmatize_text)
data_test['lemmatized_text'] = data_test['cleaned_text'].apply(lemmatize_text)

print("lemmatized_text train: \n", data_train['lemmatized_text'].head, "\n")
print("lemmatized_text test: \n", data_test['lemmatized_text'].head, "\n")

# calc tf-idf matrix
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

tfidf_matrix_train = vectorizer.fit_transform(data_train['lemmatized_text'])
tfidf_matrix_val = vectorizer.transform(data_val['lemmatized_text'])
tfidf_matrix_test = vectorizer.transform(data_test['lemmatized_text'])

# calc tf-idf matrix with uncleand data
X_train_tfidf = vectorizer.fit_transform(data_train['text'])
X_val_tfidf = vectorizer.transform(data_val['text'])
X_test_tfidf = vectorizer.transform(data_test['text'])

#print("tfidf_matrix_train_val: \n", tfidf_matrix_train, "\n")
#print("tfidf_matrix_test: \n", tfidf_matrix_test, "\n")


# count vectorizer
#X_train_vec = count_vectorizer(data_train['lemmatized_text']).toarray()
#X_val_vec = count_vectorizer(data_val['lemmatized_text']).toarray()



Shape train data:
 (34152, 2)
Shape test data:
 (9984, 2)
Cleand text train: 
 <bound method NDFrame.head of 6851     [republicans, punish, georgia, governor, refus...
17313    [father, soldier, slain, niger, defends, presi...
22435    [south, dakotas, governor, vetoes, loosening, ...
29488    [turkeys, erdogan, says, take, jerusalem, reso...
6625     [bill, maher, insults, trumps, supposed, mascu...
                               ...                        
5267     [trump, sends, hillary, pathetic, threat, one,...
13642    [breaking, fbi, investigating, clinton, crony,...
19391    [new, russian, envoy, describes, warm, meeting...
883                                         [like, father]
3939     [hilariously, amazing, video, weve, waiting, e...
Name: cleaned_text, Length: 27321, dtype: object> 

Cleand text test: 
 <bound method NDFrame.head of 0       [copycat, muslim, terrorist, arrested, assault...
1       [wow, chicago, protester, caught, camera, admi...
2       [germanys, fdp, 

In [5]:
# Naive Bayes Model
model_nb = model_naives_bayes(X_train_tfidf, data_train['label'], X_val_tfidf, data_val['label'])


Train accuracy: 0.9519783316862487

 Validation Accuracy:0.929878

classification_report:
               precision    recall  f1-score   support

           0     0.9214    0.9442    0.9327      3515
           1     0.9393    0.9147    0.9268      3316

    accuracy                         0.9299      6831
   macro avg     0.9304    0.9294    0.9298      6831
weighted avg     0.9301    0.9299    0.9298      6831



In [6]:
# Random Forest
model_rf = model_rf_train(tfidf_matrix_train, tfidf_matrix_val, data_train['label'], data_val['label'])

Random forest model
accuracy: 0.9146537842190016
Classification report:
               precision    recall  f1-score   support

           0       0.92      0.92      0.92      3515
           1       0.91      0.91      0.91      3316

    accuracy                           0.91      6831
   macro avg       0.91      0.91      0.91      6831
weighted avg       0.91      0.91      0.91      6831



In [7]:
# Multinomial Naive Bayes (MultinomialNB) classifier
model_mnb = model_multinominalNB_train(data_train['lemmatized_text'], data_val['lemmatized_text'], data_train['label'], data_val['label'])


Test
accuracy: 0.9332454984628897
Classification report:
               precision    recall  f1-score   support

           0       0.93      0.94      0.94      3515
           1       0.94      0.93      0.93      3316

    accuracy                           0.93      6831
   macro avg       0.93      0.93      0.93      6831
weighted avg       0.93      0.93      0.93      6831



In [8]:
# Simple Feedforward NN -> no good results
model_sfnn_train(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])


d:\ownCloud\Dokumente\Ironhack\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 89ms/step - accuracy: 0.8521 - loss: 1.0843 - val_accuracy: 0.5443 - val_loss: 1.3045
Epoch 2/40
107/107 ━━━━━━━━━━━━━━━━━━━━ 9s 85ms/step - accuracy: 0.9610 - loss: 0.7305 - val_accuracy: 0.5998 - val_loss: 1.1519
Epoch 3/40
107/107 ━━━━━━━━━━━━━━━━━━━━ 9s 84ms/step - accuracy: 0.9786 - loss: 0.6060 - val_accuracy: 0.7507 - val_loss: 0.9544
Epoch 4/40
107/107 ━━━━━━━━━━━━━━━━━━━━ 9s 85ms/step - accuracy: 0.9874 - loss: 0.5070 - val_accuracy: 0.8756 - val_loss: 0.7236
Epoch 5/40
107/107 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.9910 - loss: 0.4282 - val_accuracy: 0.9038 - val_loss: 0.6163
Epoch 6/40
107/107 ━━━━━━━━━━━━━━━━━━━━ 9s 85ms/step - accuracy: 0.9905 - loss: 0.3723 - val_accuracy: 0.9242 - val_loss: 0.5418
Epoch 7/40
107/107 ━━━━━━━━━━━━━━━━━━━━ 9s 81ms/step - accuracy: 0.9895 - loss: 0.3322 - val_accuracy: 0.9310 - val_loss: 0.5292
Epoch 8/40
107/107 ━━━━━━━━━━━━━━━━━━━━ 9s 83ms/step - accuracy: 0.9918 - loss: 0.2907 - val_acc

KeyboardInterrupt: 

In [ ]:
model_lr = model_logistic_regression(X_train_tfidf, data_train['label'], X_val_tfidf, data_val['label'])


Train Accuracy: 95.94%

Validation Accuracy: 93.30%

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93      3515
           1       0.92      0.95      0.93      3316

    accuracy                           0.93      6831
   macro avg       0.93      0.93      0.93      6831
weighted avg       0.93      0.93      0.93      6831



In [ ]:
## make predictions on test data
filepath = r".\data\testing_data_lowercase_labels.csv"

predict_values(model_rf, tfidf_matrix_test, data_test, filepath)

✅ Predictions saved to: .\data\testing_data_lowercase_labels.csv
